In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
# Critere de performance
def compute_pred_score(y_true, y_pred):
    y_pred_unq =  np.unique(y_pred)
    for i in y_pred_unq:
        if((i != -1) & (i!= 1) & (i!= 0) ):
            raise ValueError('The predictions can contain only -1, 1, or 0!')
    y_comp = y_true * y_pred
    score = float(10*np.sum(y_comp == -1) + np.sum(y_comp == 0))
    score /= y_comp.shape[0]
    return score


In [3]:
X_train_fname = 'training_templates.csv'
y_train_fname = 'training_labels.txt'
X_test_fname  = 'testing_templates.csv'
X_train = pd.read_csv(X_train_fname, sep=',', header=None).values
X_test  = pd.read_csv(X_test_fname,  sep=',', header=None).values
y_train = np.loadtxt(y_train_fname, dtype=np.int)

In [4]:
#import

import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, AdaBoostClassifier
#from sklearn.model_selection import cross_val_score
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import BaggingRegressor, BaggingClassifier
from sklearn import preprocessing 
#from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import PolynomialFeatures
from sklearn import linear_model
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis, LinearDiscriminantAnalysis
from sklearn.multiclass import OutputCodeClassifier
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier

In [5]:
def score_Classifier(classif):
    # RF fitting
    clf = classif
    clf.fit(X_train, y_train)

    # Prediction
    y_pred_train =  clf.predict(X_train)
    y_pred = clf.predict(X_test)

    # Compute the score
    score = compute_pred_score(y_train, y_pred_train)
    print('Score sur le train : %s' % score)

    #scores_val = cross_val_score(clf, X_train, y_train)
    #print'Cross_val score : %s' % scores_val.mean() 
    
    return y_pred

In [11]:
#MLP Classifier
start = time.time()

Y_PRED = [0]*8496
for alpha in [0.1, 0.3, 0.5, 0.7, 0.9, 1.1]:
    y_pred = score_Classifier(MLPClassifier(solver='lbfgs', alpha=alpha, hidden_layer_sizes=(5, 2), random_state=1))
    Y_PRED = np.add(Y_PRED, y_pred)

elapsed = (time.time() - start)
print('Time : %s' % elapsed)

Score sur le train : 0.474526515152
Cross_val score : 0.943020833333
Score sur le train : 0.474147727273
Cross_val score : 0.936060606061
Score sur le train : 0.471117424242
Cross_val score : 0.942130681818
Score sur le train : 0.475473484848
Cross_val score : 0.935246212121
Score sur le train : 0.475284090909
Cross_val score : 0.93665719697
Score sur le train : 0.472727272727
Cross_val score : 0.939261363636
Time : 177.71326685


In [12]:
for i in range(8496):
    if Y_PRED[i] == 6 or Y_PRED[i] == -6:
        Y_PRED[i] = Y_PRED[i]/6
    else:
        Y_PRED[i] = 0

In [12]:
# Create a random dataset
n_estimators = 10 # L in the text
tree_max_depth = 10
bagging_max_depth = 10

for n_estimators in range(10,15):
    clf = QuadraticDiscriminantAnalysis()
    clf.fit(X_train, y_train)
    bagging = BaggingClassifier(base_estimator=clf, n_estimators=n_estimators)
    bagging.fit(X_train, y_train)

    y_pred_train =  bagging.predict(X_train)
    y_pred = clf.predict(X_test)

    # Compute the score
    score = compute_pred_score(y_train, y_pred_train)
    print('Score sur le train : %s' % score)

    scores_val = cross_val_score(clf, X_train, y_train)
    print'Cross_val score : %s' % scores_val.mean() 

Score sur le train : 0.566477272727
Cross_val score : 0.9321875
Score sur le train : 0.566856060606
Cross_val score : 0.9321875
Score sur le train : 0.566098484848
Cross_val score : 0.9321875
Score sur le train : 0.564678030303
Cross_val score : 0.9321875
Score sur le train : 0.566571969697
Cross_val score : 0.9321875


In [10]:
#Quadratic Discriminant Analysis
score_Classifier(QuadraticDiscriminantAnalysis())

Score sur le train : 0.566382575758
Cross_val score : 0.9321875


array([ 1, -1,  1, ...,  1,  1,  1])

In [8]:
#k-NN 
#for n_neighbors in [75, 85]:
start = time.time()
y_pred_75 = score_Classifier(KNeighborsClassifier(n_neighbors=75, n_jobs = -1))
elapsed = (time.time() - start)
print('Time : %s' % elapsed)

start = time.time()
y_pred_85 = score_Classifier(KNeighborsClassifier(n_neighbors=85, n_jobs = -1))
elapsed = (time.time() - start)
print('Time : %s' % elapsed)

Score sur le train : 0.230303030303
Time : 2128.71899986
Score sur le train : 0.235132575758
Time : 2143.48699999


In [7]:
np.savetxt('y_pred_test.txt', y_pred, fmt='%d')

In [13]:
np.savetxt('y_pred_test.txt', Y_PRED, fmt='%d')

array([-1, -1,  1, ...,  1,  1,  1])